In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import csv
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from scipy import stats



In [ ]:
#зчитуємо данні
df_train = pd.read_csv('/kaggle/input/instant-gratification/train.csv')
df_test = pd.read_csv('/kaggle/input/instant-gratification/test.csv')

In [ ]:
#виведемо данні df_train
print('Rows and column: ', df_train.shape)
df_train.head(10)

In [ ]:
#виведемо данні df_test
print('Rows and column:', df_test.shape)
df_test.head(10)

In [ ]:
#тип данних
df_train.dtypes.value_counts()


In [ ]:
#Перевіряємо типи вхідних даних
print(df_train.dtypes.unique())
print(df_test.dtypes.unique())

In [ ]:
f = plt.figure()
sns.countplot(df_train['target'])
plt.title("Count plot",fontsize=20)
plt.show()

In [ ]:
def missing_value_count(train_set):
    rows,N = train_set.shape
    missing_value = train_set.isnull().sum()
    for i in range(N):
        if missing_value[i] > 0:
            miss_persent = missing_value[i]/rows*100
            print(i,missing_value[i],f":\t %.2f" % miss_persent,"% \t",train_set.columns[i])
    
    print(train_set.shape)

In [ ]:
df_train.isnull().sum()


In [ ]:
missing_value_count(df_train)

In [ ]:
#Візуалізуємо дані
features = list(df_train.columns)
df_train[features].hist(figsize=(16,20),color = 'g',xlabelsize=0,ylabelsize=0)

In [ ]:
#робимо нову змінну до якої записуємо чисельні значення
features = df_train.select_dtypes(include = ['float64', 'int64']).columns
fig, axes = plt.subplots(nrows=len(features), ncols=1)
fig.set_size_inches(20, 200)
for i, col in enumerate(features):
    sns.boxplot(df_train[col], ax=axes[i])
plt.show()

In [ ]:
#видаляємо колоку за id
df_train.drop('id', axis='columns', inplace=True)

In [ ]:
#видаляємо target, створюємо логістичне регресію
train_target = df_train['target']
lg_result = LogisticRegression()
df_train.drop('target', axis='columns', inplace=True)
df_train.head()

In [ ]:
df_train.head()

In [ ]:
#робимо розбиття тренувального файлу
train_result_x, test_result_x, train_result_y, test_result_y = train_test_split(df_train, train_target, test_size=0.2)

#Робимо стандартизацію
sc = StandardScaler()
scaled_train_result_x = sc.fit_transform(train_result_x)
scaled_test_result_x = sc.transform(test_result_x)

#Навчаємо модель
lg_result = LogisticRegression()
lg_result.fit(scaled_train_result_x, train_result_y)
#Випробовуємо модель
train_preds_result = lg_result.predict_proba(scaled_train_result_x)
test_preds_result = lg_result.predict_proba(scaled_test_result_x)
train_preds_result = train_preds_result[:, 1]
test_preds_result = test_preds_result[: , 1]
#Виводимо Результи навчання
print(f"Train ROC_AUC: {roc_auc_score(train_result_y, train_preds_result)}")
print(f"Test ROC_AUC: {roc_auc_score(test_result_y, test_preds_result)}")

In [ ]:
df_train.head()

In [ ]:
#беремо test щоб зробити пердбачення
df_result_test = df_test[list(df_train)].copy()
df_result_test.isnull().sum()

In [ ]:
Final = sc.fit_transform(df_result_test)
last_result = lg_result.predict_proba(Final)
last_result = last_result[: , 1]
my_submission = pd.DataFrame({'id' : df_test.id, 'target' : last_result})
my_submission.to_csv('submission.csv', index = False)

In [ ]:
my_submission.head()

In [ ]:
my_submission.describe()